In [ ]:
import pathlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import guitarpro as pygp
import glob
from aux_fun import ultimate_get_filename_2

path_to_general_read_folder = "..\..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\..\data\BGTG\BGTG-v0.1"


In [6]:
df_rg = pd.read_csv('..\\..\\data\\function_guitar-ext-with-predictions.csv')
df_rg = df_rg[['Fichier', 'Track_Name', 'Partie', 'Instrument', 'Mesure', 'rg-estimation']]
display(df_rg.head())


,Fichier,Track_Name,Partie,Instrument,Mesure,rg-estimation
0,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07
1,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,2,4.580104e-07
2,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,3,6.601997e-06
3,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,4,4.111985e-04
4,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,5,4.352387e-05


In [7]:
DADAGPPATH = pathlib.Path(path_to_general_read_folder)

all_paths = list(DADAGPPATH.rglob('**/*.pygp.gp5'))
# retrieve ALL filenames in DadaGP 
all_filenames = [f.name for f in all_paths]
all_filenames_no_ext = [f.name[:-13] for f in all_paths]

index = np.random.randint(0, len(all_paths))

all_paths[index], all_filenames[index], all_filenames_no_ext[index]
series_paths = pd.Series(all_paths)
series_filenames = pd.Series(all_filenames_no_ext)

In [8]:
tqdm.pandas()
# df.reset_index(drop=True, inplace=True)
# drop Unnamed: 0
df_to_process = df_rg.drop_duplicates(subset=['Fichier']).copy()[['Fichier', 'Partie']]
df_to_process.loc[:, 'File_Name'] = 'Not found'
df_to_process.loc[:, 'Dadagp_Path'] = 'Not found'
df_to_process.head()

,Fichier,Partie,File_Name,Dadagp_Path
0,../data/DadaGP8-gpif/ - Isolated.gpif,E. Ardanuy,Not found,Not found
131,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 1....,Trilha 1,Not found,Not found
341,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 2....,Trilha 2,Not found,Not found
386,../data/DadaGP8-gpif/*.gpif,E. Ardanuy,Not found,Not found
517,../data/DadaGP8-gpif/1 God - Grace.gpif,Track 1,Not found,Not found


In [9]:
# df_to_process = df_to_process.progress_apply(ultimate_get_filename,
#                                              axis=1, args=(all_filenames_no_ext, all_paths))
# 2min with lists, 7min with Series for function 1
df_to_process = df_to_process.progress_apply(ultimate_get_filename_2,
                                             axis=1, args=(series_filenames, series_paths))
# ~5 hours if we have to check the parts... (function 2)

  1%|          | 491/56574 [00:45<1:27:02, 10.74it/s]


KeyboardInterrupt: 

In [ ]:
# Merge back
df_rg = df_rg.merge(df_to_process[['Fichier', 'File_Name', 'Dadagp_Path']], on='Fichier', how='left')

In [ ]:
# Count number of errors
display(df_rg[df_rg['Dadagp_Path'] == 'error']['Fichier'].unique())
# Windows doesn't accept files starting with a blank space, we have to rename those

# Remove the duplicates from the dataframe
df_rg = df_rg[df_rg['Dadagp_Path'] != 'error']

array(['../data/DadaGP8-gpif/*.gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (1).gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (2).gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (3).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (1).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (2).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (3).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (1).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (2).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (3).gpif',
       '../data/DadaGP8-gpif/Irish - Kingfair (bass Solo Version) (1).gpif',
       '../data/DadaGP8-gpif/Joel, Billy - Just the way you are (1).gpif',
       '../data/DadaGP8-gpif/Kaukonen, Jorma - Hesitation Blues (1).gpif',
       '../data/DadaGP8-gpif/Saez - MontCe LO Haut (2) (1).gpif',
       '.

In [ ]:
display(df_rg.head())
df_rg.shape # 1358 measures removed because of the duplicates (very little)

,Fichier,Track_Name,Partie,Instrument,Mesure,Has_SingleNotes,Has_Chords,Has_2_note_chords,Has_3_note_chords,Has_4_note_chords,...,Letring100pourcents,Nb_VibratoW,Presence_VibratoMG,Nb_Whammy_Bar,Presence_bends,has_triad,has_fifth,rg-estimation,File_Name,Dadagp_Path
0,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,1,1,1,0,1,0,...,0,0,0,0,0,1,1,2.570728e-07,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
1,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,2,1,1,0,1,0,...,0,0,0,0,0,1,1,4.580104e-07,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
2,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,3,1,1,0,1,0,...,0,0,0,0,0,1,1,6.601997e-06,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
3,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,4,1,1,0,1,0,...,0,0,0,0,0,1,1,4.111985e-04,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...
4,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,5,1,1,0,1,0,...,0,0,0,0,0,1,1,4.352387e-05,- Isolated,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...


(4164431, 41)

In [ ]:
df_rg.to_csv('../../data/df_rg_1.csv', index=False)
display(df_rg.head())
df_rg['File_Name'].nunique()

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation
0,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07
1,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,2,4.580104e-07
2,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,3,6.601997e-06
3,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,4,4.111985e-04
4,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,5,4.352387e-05
